In [1]:
import keras
import tensorflow as tf
from keras import layers,Model
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout,InputLayer,Flatten,Dense,BatchNormalization,MaxPooling2D,Conv2D,Input,Concatenate,LeakyReLU
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical,normalize
from keras.applications.xception import Xception,preprocess_input
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint,TensorBoard
import numpy as np
import joblib
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm_notebook as tqdm
import warnings
import keras.backend as K
import random
warnings.filterwarnings('ignore')

In [2]:
#Loading already preprocessed files.
x_train=joblib.load('D:\\Py files\\AAA - Thesis Codes\\packages\\T\\x_train.pkl')
x_cv=joblib.load('D:\\Py files\\AAA - Thesis Codes\\packages\\T\\x_cv.pkl')
x_test=joblib.load('D:\\Py files\\AAA - Thesis Codes\\packages\\T\\x_test.pkl')

y_train=joblib.load('D:\\Py files\\AAA - Thesis Codes\\packages\\T\\y_train.pkl')
y_cv=joblib.load('D:\\Py files\\AAA - Thesis Codes\\packages\\T\\y_cv.pkl')
y_test=joblib.load('D:\\Py files\\AAA - Thesis Codes\\packages\\T\\y_test.pkl')

In [3]:
print("Shape of Train data {}".format(x_train.shape))
print("Shape of Cv data {}".format(x_cv.shape))
print("Shape of Test data {}".format(x_test.shape))

Shape of Train data (6840, 299, 299, 3)
Shape of Cv data (450, 299, 299, 3)
Shape of Test data (1800, 299, 299, 3)


In [4]:
Y_train=to_categorical(y_train)
Y_test = to_categorical(y_test)
Y_cv = to_categorical(y_cv)

In [5]:
print("Shape of Train data {}".format(Y_train.shape))
print("Shape of Cv data {}".format(Y_cv.shape))
print("Shape of Test data {}".format(Y_test.shape))
print(type(x_train))
print(type(Y_train))

Shape of Train data (6840, 2)
Shape of Cv data (450, 2)
Shape of Test data (1800, 2)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [6]:
#Greedy Layer-Wise Pretraining Learned from : https://machinelearningmastery.com/greedy-layer-wise-pretraining-tutorial/
# 1. Pre-training the network with train data on initilizing the "imagenet" weights with "include_top=False"
# 2. After training 3 epochs remove the top most layer
# 3. Now add similar top layer and again train and validate


Xception_initial=Xception(include_top=False,
                 weights='imagenet',
                 input_shape=(299,299,3),pooling ='avg',
                 )
#print(Xception_pre_trained.summary())

for layer in Xception_initial.layers:
    layer.trainable = True

x = Xception_initial.output
predicted = Dense(2,activation ='softmax')(x)
model_pretrain = Model(inputs = Xception_initial.input, outputs = predicted)

opt = tf.keras.optimizers.Adam(learning_rate=0.0002)
model_pretrain.compile(loss=keras.losses.categorical_crossentropy,
          optimizer=opt,
          metrics=['accuracy'])
pretraining_Xception =model_pretrain.fit(x_train, Y_train,
                                              verbose=1,
                                              batch_size=32,  
                                              epochs=3)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.